# ニュース記事のタイトル生成（推論のみ）

転移学習済みT5モデルを用いて、タイトル生成を実行します。

- **T5（Text-to-Text Transfer Transformer）**: テキストを入力されるとテキストを出力するという統一的枠組みで様々な自然言語処理タスクを解く深層学習モデル（[日本語解説](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html)）  
<img src="https://1.bp.blogspot.com/-89OY3FjN0N0/XlQl4PEYGsI/AAAAAAAAFW4/knj8HFuo48cUFlwCHuU5feQ7yxfsewcAwCLcBGAsYHQ/s1600/image2.png">  
出典: [Exploring Transfer Learning with T5: the Text-To-Text Transfer Transformer](https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html)
- **事前学習**: 個別のタスク用に学習をする前に文法や一般的な言葉の文脈的意味を学習させること（自己教師あり学習とWikipedia等の大規模データ（コーパス）を用いることで広く一般的な知識を持ったモデルを作れる）
- **転移学習、ファインチューニング**: 事前学習済みモデルを初期値にして、特定のタスク用に追加で学習を行うこと（主に教師あり学習）

入出力が次の形式を持ったタスク用に転移学習されたモデルを用いて推論を実行します。

- **入力**: "{body}"をトークナイズしたトークンID列（最大512トークン）
- **出力**: "{title}"をトークナイズしたトークンID列（最大64トークン）

ここで、{title}はニュース記事のタイトル、{body}は本文、{genre_id}はニュースの分類ラベル（0〜8）です。


# ライブラリやデータの準備

## 依存ライブラリのインストール

In [1]:
!pip install -qU torch==2.0.* torchtext==0.15.* torchvision==0.15.* torchaudio==2.0.* torchmetrics==0.11.* torchdata==0.6.* \
    transformers==4.26.1 pytorch_lightning==1.9.3 sentencepiece==0.1.97

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00


In [2]:
!pip list | grep -e "torch" -e "transformers" -e "sentencepiece"

pytorch-lightning                1.9.3
sentencepiece                    0.1.97
torch                            2.0.1+cu118
torchaudio                       2.0.2+cu118
torchdata                        0.6.1
torchmetrics                     0.11.4
torchsummary                     1.5.1
torchtext                        0.15.2
torchvision                      0.15.2+cu118
transformers                     4.26.1


## 各種ディレクトリ作成

* data: テスト用データセット格納用

In [3]:
!mkdir -p /content/data

In [4]:
# 転移学習済みモデル
MODEL_DIR = "sonoisa/t5-base-japanese-title-generation"

## livedoor ニュースコーパスのダウンロード

In [5]:
!wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2023-07-20 11:14:29--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  4.40MB/s    in 1.9s    

2023-07-20 11:14:31 (4.40 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



## livedoorニュースコーパスの形式変換

livedoorニュースコーパスを次の形式のTSVファイルに変換します。

* 1列目: タイトル
* 2列目: 本文
* 3列目: ジャンルID（0〜8）

TSVファイルは/content/dataに格納されます。


## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [6]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 情報抽出

ニュース記事のタイトルと本文とジャンル（9分類）の情報を抽出します。

In [7]:
import tarfile
import re

target_genres = ["dokujo-tsushin",
                 "it-life-hack",
                 "kaden-channel",
                 "livedoor-homme",
                 "movie-enter",
                 "peachy",
                 "smax",
                 "sports-watch",
                 "topic-news"]

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body

genre_files_list = [[] for genre in target_genres]

all_data = []

with tarfile.open("ldcc-20140209.tar.gz") as archive_file:
    for archive_item in archive_file:
        for i, genre in enumerate(target_genres):
            if genre in archive_item.name and archive_item.name.endswith(".txt"):
                genre_files_list[i].append(archive_item.name)

    for i, genre_files in enumerate(genre_files_list):
        for name in genre_files:
            file = archive_file.extractfile(name)
            title, body = read_title_body(file)
            title = normalize_text(title)
            body = normalize_text(body)

            if len(title) > 0 and len(body) > 0:
                all_data.append({
                    "title": title,
                    "body": body,
                    "genre_id": i
                    })

## データ分割

データセットを90% : 5%: 5% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [8]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    title = data["title"]
    body = data["body"]
    genre_id = data["genre_id"]

    assert len(title) > 0 and len(body) > 0
    return f"{title}\t{body}\t{genre_id}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05

with open(f"data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"data/dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"data/test.tsv", "w", encoding="utf-8") as f_test:

    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

7334it [00:00, 60642.85it/s]


作成されたデータを確認します。

形式: {タイトル}\t{本文}\t{ジャンルID}

In [9]:
!head -3 data/test.tsv

別れる?続ける?3月末で決断を迫る査定サービス	3月に入り、世間は卒業シーズン真っ只中。誰もが経験したことのある“別れ"の季節だけに寂しさも感じるが、その先にある新たな出会いにも心躍らせる季節ともいえる。社会人でも転勤や異動など、人の入れ替わりや案件の整理など、とにかく「忙しい」というイメージが先行してしまいがちなのが年度末。さらには、予算消化の道路工事が各所で行われる光景もまた、気忙しくさせる原因でもある。年度末といえば、課税標準の時期でもある。住民税も所得税も車両税も、すべてその年の3月末を起点として算出される。つまり「クルマを持っているとお金が掛かる」ことを最も実感する時期でもあるのだ。そしてクルマ関連でもうひとつ、年度末のこの時期にこそ注目してほしいことがある。それは「年度末はクルマの売買が最も盛んな時期」ということ。下記のグラフを見れば一目瞭然の事実。3月の自動車登録台数は、他の月の2倍近い数字を記録している。これは、先述のとおり課税時期であること、そして自動車ディーラー等で大規模なキャンペーンを行うためだ。こんな時期だからこそ、クルマ乗換ようかな?売ろうかな?と悩む人も多いだろう。その前に、まずは愛車の買取相場を調べてみてはいかがだろうか?愛車が今いくらなのか、現在価格を正確に知っておことは、この重要な時期の大切な情報と言える。そこで、オススメするのは、日本最大規模の中古車査定ネットワークを誇るカービューの「愛車無料査定」だ。この「愛車無料査定」は、無料サービスでありながら、業界最大規模の査定データベースによって、車の買取相場がオンライン上で分かると同時に、複数社の買取店に見積りを依頼することが出来る。インターネットからサイトにアクセスし査定したい車種と郵便番号、さらには車種情報などを入力すれば、登録地域の買取店、最大8社の査定を比較することが出来る。申込み時間は約3分程度と、とても便利なサービスなのだ。3月という節目の季節。愛車とも「そろそろお別れか」と考えてみても、判断が難しいだろう。悩む前に愛車の現在価格を調べることをオススメする。・あなたの愛車、今いくら?webで買取相場をチェック!・自動車総合サイトcarview.co.jp「愛車無料査定」	3
イベント作成や友人の紹介方法facebook活用のスゴ技・裏技テクニック集	最近では仕事での付き合い

# 推論に必要なクラス等の定義

推論にはPyTorch/Transformersを利用します。

In [10]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [11]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    model_name_or_path=MODEL_DIR,
    tokenizer_name_or_path=MODEL_DIR,

    max_input_length=512,
    max_target_length=64,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{title}\t{body}\t{genre_id}"です。

In [12]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)

        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask,
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, title, body, genre_id):
        # ニュースタイトル生成タスク用の入出力形式に変換する。
        input = f"{body}"
        target = f"{title}"
        return input, target

    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 3
                assert len(line[0]) > 0
                assert len(line[1]) > 0
                assert len(line[2]) > 0

                title = line[0]
                body = line[1]
                genre_id = line[2]

                input, target = self._make_record(title, body, genre_id)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [13]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "train.tsv",
                           input_max_len=args_dict["max_input_length"],
                           target_max_len=args_dict["max_target_length"])

テストデータの1レコード目をみてみます。

In [14]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
huluは昨年9月にサービスを開始して以来、日本でのビジネスを順調に拡大してきた。現在は約1000本の映画と、10,000話以上に及ぶテレビ番組の視聴が可能だ。そんなhuluから新たなニュースが飛び込んできた。 ■今日からapple tvに対応!今日、9月4日から新たに「apple tv」にてhuluを楽しめるようになったという。aple idを用いて新規登録や決済も可能だという。apple tvのホーム画面からhuluを起動し、すでにアカウントを持っているユーザーはログイン。また、まだアカウントを持っていない人も簡単にitunesアカウントで登録できる。apple tvより新規登録したユーザーは通常2週間の無料トライアルが1ヶ月になるということだ。 ■サービス一周年記念「huluアンバサダープロジェクト」9月12日からサービス一周年を記念したスペシャルプロジェクト「huluアンバサダープロジェクト」が実施される。facebookやtwitterを通してhuluについて情報発信すると「huluアンバサダー」に任命され、先着10,000名の新規登録者は1ケ月無料トライアルが楽しめるということだ。コンテンツも対応デバイスもますます増えるhuluに今後も注目だ。</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

# 学習済みモデルの読み込み

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# 全テストデータの本文に対するタイトル生成

In [16]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv",
                          input_max_len=args_dict["max_input_length"],
                          target_max_len=args_dict["max_target_length"])

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids,
        attention_mask=input_mask,
        max_length=args_dict["max_target_length"],
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True,
                            clean_up_tokenization_spaces=False)
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/46 [00:00<?, ?it/s]

## 生成結果確認

形式
- generated: 生成されたタイトル
- actual: 人が作成したタイトル（正解）
- body: ニュース記事の本文


In [17]:
for output, target, input in zip(outputs, targets, inputs):
    print("generated: " + output)
    print("actual:    " + target)
    # print("body:      " + input)
    print()

generated: あなたの愛車が今いくら?オンライン上で一括査定できる「愛車無料査定」
actual:    別れる?続ける?3月末で決断を迫る査定サービス

generated: facebook活用術/同窓会・女子会で大活躍!など-【知っ得!虎の巻】
actual:    イベント作成や友人の紹介方法facebook活用のスゴ技・裏技テクニック集

generated: nttドコモ、android 4.0 ics搭載3.4インチスマホ「aquos phone st sh-07d」を発表!1ghzcpuや防水、ワンセグに対応
actual:    nttドコモ、android 4.0 ics搭載3.4インチスマホ「aquos phone st sh-07d」を発表!1ghzcpuや防水、防塵、ワンセグに対応

generated: プロ野球ファンの想いを数値化して競う、今年の“目利き指数"とは?
actual:    2010年プロ野球界のキーマンは?

generated: インタビュー:森下奈保子さん「映画を創る、女性たち。」
actual:    「映画祭を創る、女性たち。」vol.4コンテンツマーケットディレクター森下美香さん

generated: タンスの肥やしがドンドン増える!?宅配買取サービス『ブランディア』
actual:    タンスの肥やしがサイフの糧に!?最新の買取りサービスの実力とは

generated: ダース・ベーダー風のデザインがかわいい!『kinectスター・ウォーズ』が数量限定発売
actual:    スターウォーズ「r2-d2」をイメージした限定xbox発売決定!ダサい?それともレトロでかわいい?

generated: なでしこ田中陽子の活躍に期待!?
actual:    期待高まる、澤穂希の“美少女後継者"

generated: 橋下徹大阪府知事が週刊新潮の取材批判にツイッターで賛否両論
actual:    ツイッター上で週刊新潮にブチギレした橋下徹氏にネットでは賛否両論

generated: 神戸コレクション ′10s/s写真集vol.2
actual:    加藤夏希、マリエによる“おんなのこ革命。" 神戸コレクション ′10s/s

generated: ニコニコ動画、新ui(ユーザーインターフェイ

# 任意の文章に対するタイトル生成

文章に合うタイトルを10個、自動生成してみます。

以下のコードではタイトルの多様性を生むために色々generateメソッドのパラメータを設定しています。パラメータの詳細は下記リンク先を参照してください。

- [generateメソッドのパラメータの意味](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

In [18]:
# 文章の出典: https://qiita.com/sonoisa/items/cf0bc6c0ed4d244407b4
# 正解: LEGOで作るスマートロック　〜「Hey Siri 鍵開けて」を実現する方法 〜
body = """
これはLEGOとRaspberry Piで実用的なスマートロックを作り上げる物語です。
スマートロック・システムの全体構成は下図のようになります。図中右上にある塊が、全部LEGOで作られたスマートロックです。

特徴は、3Dプリンタ不要で、LEGOという比較的誰でも扱えるもので作られたハードウェアであるところ、見た目の野暮ったさと機能のスマートさを兼ね備え、エンジニア心をくすぐるポイント満載なところです。
なお、LEGO (レゴ)、LEGO Boost (ブースト) は LEGO Group (レゴグループ) の登録商標であり、この文書はレゴグループやその日本法人と一切関係はありません。

次のようなシチュエーションを経験したことはありませんか？

- 外出先にて、「そういや、鍵、閉めてきたかな？記憶がない…（ソワソワ）」
- 朝の通勤にて、駅に到着してみたら「あ、鍵閉め忘れた。戻るか…」
- 料理中に「あ、鍵閉め忘れた！でも、いま手が離せない。」
- 玄関先で「手は買い物で一杯。ポケットから鍵を出すのが大変。」
- 職場にて、夕方「そろそろ子供は家に帰ってきたかな？」
- 玄関にて「今日は傘いるかな？」

今回作るスマートロックは、次の機能でこれらを解決に導きます。

- 鍵の閉め忘れをSlackに通知してくれる。iPhoneで施錠状態を確認できる。
- 何処ででもiPhoneから施錠できる。
- 「Hey Siri 鍵閉めて（鍵開けて）」で施錠/開錠できる。
- 鍵の開閉イベントがiPhoneに通知され、帰宅が分かる。
- LEDの色で天気予報（傘の必要性）を教えてくれる（ただし、時間の都合で今回は説明省略）。

欲しくなりましたでしょうか？

以下、ムービー多めで機能の詳細と作り方について解説していきます。ハードウェアもソフトウェアもオープンソースとして公開します。
"""

In [19]:
MAX_SOURCE_LENGTH = args_dict["max_input_length"]   # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = args_dict["max_target_length"]  # 生成されるタイトルの最大トークン数

def preprocess_body(text):
    return normalize_text(text.replace("\n", " "))

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_body(body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True,
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()

# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask,
    max_length=MAX_TARGET_LENGTH,
    temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
    num_beams=10,             # ビームサーチの探索幅
    diversity_penalty=1.0,    # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,       # ビームサーチのグループ数
    num_return_sequences=10,  # 生成する文の数
    repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True,
                                     clean_up_tokenization_spaces=False)
                    for ids in outputs]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
    print(f"{i+1:2}. {title}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


 1. アイデア満載の「lego」で作るスマートロックの作り方
 2. 「レゴ」とraspberry piで作る実用的なスマートロック
 3. 「レゴ」とraspberry piで作る実用的なスマートロック(下)
 4. 開発秘話!legoで作る実用的なスマートロック(下)
 5. アイディアとアイデアで作る、legoで作られたスマートロックの作り方
 6. アイディアとアイデアで作る、legoで作られたスマートロック
 7. “lego"で作る実用的なスマートロック、いかがでしたでしょうか?
 8. デザインが野暮ったい!誰でも扱えるスマートロックを作ろう
 9. アイディアとアイデアで作る、legoで作られたスマートロックの作り方(下)
10. デザインが野暮ったい!誰でも扱えるスマートロックを作ろう!
